In [1]:
import numpy as np 
import pandas as pd 

import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)

from scipy import stats



In [2]:
in_data_a = pd.read_csv('../input/200114-test-work-data/in_data_a.csv')
in_data_p = pd.read_csv('../input/200114-test-work-data/in_data_p.csv')

In [3]:
# проверяем, что нет пустых элементов в данных

in_data_a.isnull().sum()

Date        0
Campaign    0
id          0
ad_id       0
os          0
Installs    0
app         0
dtype: int64

In [4]:
in_data_p.isnull().sum()

date        0
campaign    0
ad_id       0
spend       0
dtype: int64

In [5]:
# смотрим важные параметры-суммы, которые будем проверять в конце, чтоб бились

in_data_a.Installs.sum()

18524.0

In [6]:
in_data_p.spend.sum()

1651202.4700000002

In [7]:
in_data_a.os.value_counts()

android    36189
ios        17065
server      2303
windows      292
linux          8
macos          6
unknown        6
Name: os, dtype: int64

In [8]:
# скользко в среднем за установку по всем данным

in_data_p.spend.sum() / in_data_a.Installs.sum()

89.13854836968258

In [9]:
in_data_a_modified = in_data_a

In [10]:
# добавляем столбец с новым ключом (date и ad_id)

in_data_a_modified.loc[:,'date_ad_id'] = in_data_a_modified.loc[:,'Date'].add(in_data_a_modified.loc[:,'ad_id'].astype(str))

In [11]:
in_data_a_modified

,Date,Campaign,id,ad_id,os,Installs,app,date_ad_id
0,2019-03-01,MSK,1190,36296008,ios,0.0,real,2019-03-0136296008
1,2019-03-01,MSK,1190,36296012,ios,0.0,real,2019-03-0136296012
2,2019-03-01,MSK,1190,36296013,ios,0.0,real,2019-03-0136296013
3,2019-03-01,MSK,1190,36296022,ios,0.0,real,2019-03-0136296022
4,2019-03-01,MSK,1190,36296023,ios,0.0,real,2019-03-0136296023
...,...,...,...,...,...,...,...,...
55864,2019-03-11,SPB,2260,35102012,ios,0.0,auto,2019-03-1135102012
55865,2019-03-11,SPB,2260,35102013,ios,0.0,auto,2019-03-1135102013
55866,2019-03-11,SPB,2260,35102014,ios,0.0,auto,2019-03-1135102014
55867,2019-03-11,SPB,2260,37165456,ios,0.0,auto,2019-03-1137165456


In [12]:
duplicates = in_data_a_modified[ in_data_a_modified.duplicated(['date_ad_id']) ]

In [13]:
# смотрим, что в дубликатах нет инсталов:

duplicates.Installs.sum()

0.0

In [14]:
# смотрим сколько первостепенных осей в инсталах:

duplicates.os.value_counts()

server     2290
windows     265
linux         8
unknown       6
macos         5
ios           3
Name: os, dtype: int64

In [ ]:
# три штучки ios и те с нулевым количеством инсталов,- можно дропать, все затраты запишем на android в этих трех случаях

In [15]:
dropped_duplicates = in_data_a_modified.merge(duplicates, how='left', indicator=True)
dropped_duplicates

,Date,Campaign,id,ad_id,os,Installs,app,date_ad_id,_merge
0,2019-03-01,MSK,1190,36296008,ios,0.0,real,2019-03-0136296008,left_only
1,2019-03-01,MSK,1190,36296012,ios,0.0,real,2019-03-0136296012,left_only
2,2019-03-01,MSK,1190,36296013,ios,0.0,real,2019-03-0136296013,left_only
3,2019-03-01,MSK,1190,36296022,ios,0.0,real,2019-03-0136296022,left_only
4,2019-03-01,MSK,1190,36296023,ios,0.0,real,2019-03-0136296023,left_only
...,...,...,...,...,...,...,...,...,...
55864,2019-03-11,SPB,2260,35102012,ios,0.0,auto,2019-03-1135102012,left_only
55865,2019-03-11,SPB,2260,35102013,ios,0.0,auto,2019-03-1135102013,left_only
55866,2019-03-11,SPB,2260,35102014,ios,0.0,auto,2019-03-1135102014,left_only
55867,2019-03-11,SPB,2260,37165456,ios,0.0,auto,2019-03-1137165456,left_only


In [16]:
# дропаем ненужные дубликаты

dropped_duplicates = dropped_duplicates[dropped_duplicates['_merge'] == 'left_only']
dropped_duplicates

,Date,Campaign,id,ad_id,os,Installs,app,date_ad_id,_merge
0,2019-03-01,MSK,1190,36296008,ios,0.0,real,2019-03-0136296008,left_only
1,2019-03-01,MSK,1190,36296012,ios,0.0,real,2019-03-0136296012,left_only
2,2019-03-01,MSK,1190,36296013,ios,0.0,real,2019-03-0136296013,left_only
3,2019-03-01,MSK,1190,36296022,ios,0.0,real,2019-03-0136296022,left_only
4,2019-03-01,MSK,1190,36296023,ios,0.0,real,2019-03-0136296023,left_only
...,...,...,...,...,...,...,...,...,...
55864,2019-03-11,SPB,2260,35102012,ios,0.0,auto,2019-03-1135102012,left_only
55865,2019-03-11,SPB,2260,35102013,ios,0.0,auto,2019-03-1135102013,left_only
55866,2019-03-11,SPB,2260,35102014,ios,0.0,auto,2019-03-1135102014,left_only
55867,2019-03-11,SPB,2260,37165456,ios,0.0,auto,2019-03-1137165456,left_only


In [17]:
# проверяем, что суммы строк бьются

print( len(in_data_a), len(dropped_duplicates) + len(duplicates) )

55869 55869


In [18]:
# проверяем, что суммы инсталлов сохранились

print(in_data_a.Installs.sum(), dropped_duplicates.Installs.sum())

18524.0 18524.0


In [19]:
# проверяем, что суммы основных осей сохранились (кроме тех трех штук ios в дубликатах)

print(in_data_a.os.value_counts())

print(dropped_duplicates.os.value_counts())

android    36189
ios        17065
server      2303
windows      292
linux          8
macos          6
unknown        6
Name: os, dtype: int64
android    36189
ios        17062
windows       27
server        13
macos          1
Name: os, dtype: int64


In [ ]:
# сохранилась часть второстепенных осей, но они сохранились, тк были не в дубликатах и возможно туда запишутся траты

In [25]:
# добавляем столбец с новым ключом (date и ad_id) к таблице in_data_p

in_data_p_modified = in_data_p
in_data_p_modified.loc[:,'date_ad_id'] = in_data_p_modified.loc[:,'date'].add(in_data_p_modified.loc[:,'ad_id'].astype(str))
in_data_p_modified

,date,campaign,ad_id,spend,date_ad_id
0,2019-02-11,11484480,32631604,49.18,2019-02-1132631604
1,2019-02-11,11484485,32631612,33.19,2019-02-1132631612
2,2019-02-11,11484611,32631778,2.52,2019-02-1132631778
3,2019-02-11,11487692,32636729,7.08,2019-02-1132636729
4,2019-02-11,11487692,32636730,7.47,2019-02-1132636730
...,...,...,...,...,...
19699,2019-03-11,15261292,38516327,0.00,2019-03-1138516327
19700,2019-03-11,15261302,38516337,0.00,2019-03-1138516337
19701,2019-03-11,15261303,38516338,0.00,2019-03-1138516338
19702,2019-03-11,15261304,38516339,0.00,2019-03-1138516339


In [26]:
# мержим таблицы

merged = pd.merge(dropped_duplicates, in_data_p_modified, how='outer', on="date_ad_id")
merged['Installs'] = merged['Installs'].fillna(0)
merged['spend'] = merged['spend'].fillna(0)

In [27]:
merged

,Date,Campaign,id,ad_id_x,os,Installs,app,date_ad_id,_merge,date,campaign,ad_id_y,spend
0,2019-03-01,MSK,1190.0,36296008.0,ios,0.0,real,2019-03-0136296008,left_only,NaN,NaN,NaN,0.0
1,2019-03-01,MSK,1190.0,36296012.0,ios,0.0,real,2019-03-0136296012,left_only,NaN,NaN,NaN,0.0
2,2019-03-01,MSK,1190.0,36296013.0,ios,0.0,real,2019-03-0136296013,left_only,NaN,NaN,NaN,0.0
3,2019-03-01,MSK,1190.0,36296022.0,ios,0.0,real,2019-03-0136296022,left_only,NaN,NaN,NaN,0.0
4,2019-03-01,MSK,1190.0,36296023.0,ios,0.0,real,2019-03-0136296023,left_only,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57566,NaN,NaN,NaN,NaN,NaN,0.0,NaN,2019-03-1138516288,NaN,2019-03-11,15261272.0,38516288.0,0.0
57567,NaN,NaN,NaN,NaN,NaN,0.0,NaN,2019-03-1138516324,NaN,2019-03-11,15261289.0,38516324.0,0.0
57568,NaN,NaN,NaN,NaN,NaN,0.0,NaN,2019-03-1138516325,NaN,2019-03-11,15261290.0,38516325.0,0.0
57569,NaN,NaN,NaN,NaN,NaN,0.0,NaN,2019-03-1138516326,NaN,2019-03-11,15261291.0,38516326.0,0.0


In [28]:
# проверка

print('Installs before:',in_data_a.Installs.sum(), '\n', 'Installs after:', merged.Installs.sum(), '\n')
print('Spent before:',in_data_p.spend.sum(), '\n', 'Spent after:', merged.spend.sum(), '\n')
print('Os counts before:', '\n',in_data_a.os.value_counts(), '\n')
print('Os counts after:', '\n', merged.os.value_counts(), '\n')
print('Average price for install before:',in_data_p.spend.sum() / in_data_a.Installs.sum(), '\n', 'Average price for install after:', merged.spend.sum() / merged.Installs.sum(), '\n')


Installs before: 18524.0 
 Installs after: 18524.0 

Spent before: 1651202.4700000002 
 Spent after: 1651202.47 

Os counts before: 
 android    36189
ios        17065
server      2303
windows      292
linux          8
macos          6
unknown        6
Name: os, dtype: int64 

Os counts after: 
 android    36189
ios        17062
windows       27
server        13
macos          1
Name: os, dtype: int64 

Average price for install before: 89.13854836968258 
 Average price for install after: 89.13854836968257 



In [31]:
# оставляем нужно - с инсталами или с платами

merged = merged[ ((merged.Installs == 0) & (merged.spend > 0)) | (merged.Installs > 0) ]

In [32]:
# проверка

print('Installs before:',in_data_a.Installs.sum(), '\n', 'Installs after:', merged.Installs.sum(), '\n')
print('Spent before:',in_data_p.spend.sum(), '\n', 'Spent after:', merged.spend.sum(), '\n')
print('Os counts before:', '\n',in_data_a.os.value_counts(), '\n')
print('Os counts after:', '\n', merged.os.value_counts(), '\n')
print('Average price for install before:',in_data_p.spend.sum() / in_data_a.Installs.sum(), '\n', 'Average price for install after:', merged.spend.sum() / merged.Installs.sum(), '\n')


Installs before: 18524.0 
 Installs after: 18524.0 

Spent before: 1651202.4700000002 
 Spent after: 1651202.4700000002 

Os counts before: 
 android    36189
ios        17065
server      2303
windows      292
linux          8
macos          6
unknown        6
Name: os, dtype: int64 

Os counts after: 
 android    6849
ios        2256
server        1
windows       1
Name: os, dtype: int64 

Average price for install before: 89.13854836968258 
 Average price for install after: 89.13854836968258 



In [33]:
merged

,Date,Campaign,id,ad_id_x,os,Installs,app,date_ad_id,_merge,date,campaign,ad_id_y,spend
12,2019-03-01,MSK,1190.0,36374453.0,ios,0.0,real,2019-03-0136374453,left_only,2019-03-01,14012582.0,36374453.0,4.35
13,2019-03-01,MSK,1190.0,36374509.0,ios,0.0,real,2019-03-0136374509,left_only,2019-03-01,14012618.0,36374509.0,6.28
15,2019-03-01,MSK,1190.0,36444120.0,ios,0.0,real,2019-03-0136444120,left_only,2019-03-01,14060667.0,36444120.0,4.45
16,2019-03-01,MSK,1190.0,36444121.0,ios,0.0,real,2019-03-0136444121,left_only,2019-03-01,14060667.0,36444121.0,5.63
17,2019-03-01,MSK,1190.0,36444122.0,ios,0.0,real,2019-03-0136444122,left_only,2019-03-01,14060667.0,36444122.0,1.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57484,NaN,NaN,NaN,NaN,NaN,0.0,NaN,2019-03-0637998082,NaN,2019-03-06,14949778.0,37998082.0,4.14
57496,NaN,NaN,NaN,NaN,NaN,0.0,NaN,2019-03-0737998082,NaN,2019-03-07,14949778.0,37998082.0,3.78
57528,NaN,NaN,NaN,NaN,NaN,0.0,NaN,2019-03-0837998082,NaN,2019-03-08,14949778.0,37998082.0,3.03
57536,NaN,NaN,NaN,NaN,NaN,0.0,NaN,2019-03-1037998082,NaN,2019-03-10,14949778.0,37998082.0,3.11


In [34]:
# заполняем даты, там где пустые поля (там в некоторых случаях были статы, а в таблице установок данных по этим датам не было. Установки появлялись в следуюшие дни.)
merged['Date'] = merged['Date'].fillna(merged['date'])

In [35]:
merged

,Date,Campaign,id,ad_id_x,os,Installs,app,date_ad_id,_merge,date,campaign,ad_id_y,spend
12,2019-03-01,MSK,1190.0,36374453.0,ios,0.0,real,2019-03-0136374453,left_only,2019-03-01,14012582.0,36374453.0,4.35
13,2019-03-01,MSK,1190.0,36374509.0,ios,0.0,real,2019-03-0136374509,left_only,2019-03-01,14012618.0,36374509.0,6.28
15,2019-03-01,MSK,1190.0,36444120.0,ios,0.0,real,2019-03-0136444120,left_only,2019-03-01,14060667.0,36444120.0,4.45
16,2019-03-01,MSK,1190.0,36444121.0,ios,0.0,real,2019-03-0136444121,left_only,2019-03-01,14060667.0,36444121.0,5.63
17,2019-03-01,MSK,1190.0,36444122.0,ios,0.0,real,2019-03-0136444122,left_only,2019-03-01,14060667.0,36444122.0,1.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57484,2019-03-06,NaN,NaN,NaN,NaN,0.0,NaN,2019-03-0637998082,NaN,2019-03-06,14949778.0,37998082.0,4.14
57496,2019-03-07,NaN,NaN,NaN,NaN,0.0,NaN,2019-03-0737998082,NaN,2019-03-07,14949778.0,37998082.0,3.78
57528,2019-03-08,NaN,NaN,NaN,NaN,0.0,NaN,2019-03-0837998082,NaN,2019-03-08,14949778.0,37998082.0,3.03
57536,2019-03-10,NaN,NaN,NaN,NaN,0.0,NaN,2019-03-1037998082,NaN,2019-03-10,14949778.0,37998082.0,3.11


In [36]:
# проверяем, что все ad_id относятся к одинаковым кампаниям (на всякий случай)
unique_ad_ids = pd.unique(merged['ad_id_y'].values)

for ad_id in unique_ad_ids:
    if not merged[ merged.ad_id_y == ad_id ].Campaign.nunique() == 1 and merged[ merged.ad_id_y == ad_id ].ad_id_x.nunique() == 1 and merged[ merged.ad_id_y == ad_id ].os.nunique() == 1 and merged[ merged.ad_id_y == ad_id ].app.nunique() == 1:
        print('ad_id', ad_id, 'has multiple types of values')

In [37]:
# заполняем другие пустые значения по всем отдельным кампаниям
merged_non_null = merged

for ad_id in unique_ad_ids:
    sub_df = merged[ merged.ad_id_y == ad_id ]
    if len(pd.unique( sub_df.Campaign.dropna().values )):
        merged_non_null['Campaign'] = merged_non_null['Campaign'].fillna( pd.unique( sub_df.Campaign.dropna().values )[0] )
    if len(pd.unique( sub_df.id.dropna().values )):
        merged_non_null['id'] = merged_non_null['id'].fillna( pd.unique( sub_df.id.dropna().values )[0] )
    if len(pd.unique( sub_df.ad_id_x.dropna().values )):
        merged_non_null['ad_id_x'] = merged_non_null['ad_id_x'].fillna( pd.unique( sub_df.ad_id_x.dropna().values )[0] )
    if len(pd.unique( sub_df.os.dropna().values )):
        merged_non_null['os'] = merged_non_null['os'].fillna( pd.unique( sub_df.os.dropna().values )[0] )
    if len(pd.unique( sub_df.app.dropna().values )):
        merged_non_null['app'] = merged_non_null['app'].fillna( pd.unique( sub_df.app.dropna().values )[0] )

In [38]:
merged_non_null

,Date,Campaign,id,ad_id_x,os,Installs,app,date_ad_id,_merge,date,campaign,ad_id_y,spend
12,2019-03-01,MSK,1190.0,36374453.0,ios,0.0,real,2019-03-0136374453,left_only,2019-03-01,14012582.0,36374453.0,4.35
13,2019-03-01,MSK,1190.0,36374509.0,ios,0.0,real,2019-03-0136374509,left_only,2019-03-01,14012618.0,36374509.0,6.28
15,2019-03-01,MSK,1190.0,36444120.0,ios,0.0,real,2019-03-0136444120,left_only,2019-03-01,14060667.0,36444120.0,4.45
16,2019-03-01,MSK,1190.0,36444121.0,ios,0.0,real,2019-03-0136444121,left_only,2019-03-01,14060667.0,36444121.0,5.63
17,2019-03-01,MSK,1190.0,36444122.0,ios,0.0,real,2019-03-0136444122,left_only,2019-03-01,14060667.0,36444122.0,1.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57484,2019-03-06,MSK,1190.0,36374453.0,ios,0.0,real,2019-03-0637998082,NaN,2019-03-06,14949778.0,37998082.0,4.14
57496,2019-03-07,MSK,1190.0,36374453.0,ios,0.0,real,2019-03-0737998082,NaN,2019-03-07,14949778.0,37998082.0,3.78
57528,2019-03-08,MSK,1190.0,36374453.0,ios,0.0,real,2019-03-0837998082,NaN,2019-03-08,14949778.0,37998082.0,3.03
57536,2019-03-10,MSK,1190.0,36374453.0,ios,0.0,real,2019-03-1037998082,NaN,2019-03-10,14949778.0,37998082.0,3.11


In [39]:
# проверка

print('Installs before:',in_data_a.Installs.sum(), '\n', 'Installs after:', merged_non_null.Installs.sum(), '\n')
print('Spent before:',in_data_p.spend.sum(), '\n', 'Spent after:', merged_non_null.spend.sum(), '\n')
print('Os counts before:', '\n',in_data_a.os.value_counts(), '\n')
print('Os counts after:', '\n', merged_non_null.os.value_counts(), '\n')
print('Average price for install before:',in_data_p.spend.sum() / in_data_a.Installs.sum(), '\n', 'Average price for install after:', merged_non_null.spend.sum() / merged_non_null.Installs.sum(), '\n')


Installs before: 18524.0 
 Installs after: 18524.0 

Spent before: 1651202.4700000002 
 Spent after: 1651202.4700000002 

Os counts before: 
 android    36189
ios        17065
server      2303
windows      292
linux          8
macos          6
unknown        6
Name: os, dtype: int64 

Os counts after: 
 android    6849
ios        5418
server        1
windows       1
Name: os, dtype: int64 

Average price for install before: 89.13854836968258 
 Average price for install after: 89.13854836968258 



In [40]:
# Все NaN заполнены
merged_non_null.isnull().sum()

Date             0
Campaign         0
id               0
ad_id_x          0
os               0
Installs         0
app              0
date_ad_id       0
_merge        3162
date           399
campaign       399
ad_id_y        399
spend            0
dtype: int64

In [41]:
# записываем результаты

out = pd.read_csv('../input/200114-test-work-data/out.csv')

out['app'] = merged_non_null['app']
out['date'] = merged_non_null['Date']
out['campaign'] = merged_non_null['Campaign']
out['os'] = merged_non_null['os']
out['installs'] = merged_non_null['Installs']
out['spend'] = merged_non_null['spend']

In [42]:
# проверка

print('Installs before:',in_data_a.Installs.sum(), '\n', 'Installs after:', out.installs.sum(), '\n')
print('Spent before:',in_data_p.spend.sum(), '\n', 'Spent after:', out.spend.sum(), '\n')
print('Os counts before:', '\n',in_data_a.os.value_counts(), '\n')
print('Os counts after:', '\n', out.os.value_counts(), '\n')
print('Average price for install before:',in_data_p.spend.sum() / in_data_a.Installs.sum(), '\n', 'Average price for install after:', out.spend.sum() / out.installs.sum(), '\n')


Installs before: 18524.0 
 Installs after: 18524.0 

Spent before: 1651202.4700000002 
 Spent after: 1651202.4700000002 

Os counts before: 
 android    36189
ios        17065
server      2303
windows      292
linux          8
macos          6
unknown        6
Name: os, dtype: int64 

Os counts after: 
 android    6849
ios        5418
server        1
windows       1
Name: os, dtype: int64 

Average price for install before: 89.13854836968258 
 Average price for install after: 89.13854836968258 



In [43]:
out

,app,date,campaign,os,installs,spend,cpi
12,real,2019-03-01,MSK,ios,0.0,4.35,NaN
13,real,2019-03-01,MSK,ios,0.0,6.28,NaN
15,real,2019-03-01,MSK,ios,0.0,4.45,NaN
16,real,2019-03-01,MSK,ios,0.0,5.63,NaN
17,real,2019-03-01,MSK,ios,0.0,1.94,NaN
...,...,...,...,...,...,...,...
57484,real,2019-03-06,MSK,ios,0.0,4.14,NaN
57496,real,2019-03-07,MSK,ios,0.0,3.78,NaN
57528,real,2019-03-08,MSK,ios,0.0,3.03,NaN
57536,real,2019-03-10,MSK,ios,0.0,3.11,NaN


In [44]:
# суммируем/группируем по уникальным комбинациям 'app','date','campaign','os'
out = out.groupby(['app','date','campaign','os']).sum().reset_index()
out

,app,date,campaign,os,installs,spend
0,auto,2019-02-11,MSK_SPB,android,4.0,492.87
1,auto,2019-02-11,MSK_SPB,ios,2.0,132.42
2,auto,2019-02-11,RU,android,187.0,13677.70
3,auto,2019-02-11,RU,ios,21.0,1632.38
4,auto,2019-02-11,SPB,android,4.0,251.29
...,...,...,...,...,...,...
251,real,2019-03-10,SPB,ios,13.0,1340.00
252,real,2019-03-11,MSK,android,78.0,3659.61
253,real,2019-03-11,MSK,ios,9.0,284.88
254,real,2019-03-11,SPB,android,27.0,2314.06


In [45]:
# проверка, что дублей не осталось
out[ out.duplicated(subset=['app','date','campaign','os'], keep=False) ]

,app,date,campaign,os,installs,spend


In [46]:
out['cpi'] = out['spend'] / out['installs']

In [48]:
# проверка

print('Installs before:',in_data_a.Installs.sum(), '\n', 'Installs after:', out.installs.sum(), '\n')
print('Spent before:',in_data_p.spend.sum(), '\n', 'Spent after:', out.spend.sum(), '\n')
print('Os counts before:', '\n',in_data_a.os.value_counts(), '\n')
print('Os counts after:', '\n', out.os.value_counts(), '\n')
print('Average price for install before:',in_data_p.spend.sum() / in_data_a.Installs.sum(), '\n', 'Average price for install after:', out.spend.sum() / out.installs.sum(), '\n')


Installs before: 18524.0 
 Installs after: 18524.0 

Spent before: 1651202.4700000002 
 Spent after: 1651202.47 

Os counts before: 
 android    36189
ios        17065
server      2303
windows      292
linux          8
macos          6
unknown        6
Name: os, dtype: int64 

Os counts after: 
 android    145
ios        109
server       1
windows      1
Name: os, dtype: int64 

Average price for install before: 89.13854836968258 
 Average price for install after: 89.13854836968257 



In [49]:
out

,app,date,campaign,os,installs,spend,cpi
0,auto,2019-02-11,MSK_SPB,android,4.0,492.87,123.217500
1,auto,2019-02-11,MSK_SPB,ios,2.0,132.42,66.210000
2,auto,2019-02-11,RU,android,187.0,13677.70,73.142781
3,auto,2019-02-11,RU,ios,21.0,1632.38,77.732381
4,auto,2019-02-11,SPB,android,4.0,251.29,62.822500
...,...,...,...,...,...,...,...
251,real,2019-03-10,SPB,ios,13.0,1340.00,103.076923
252,real,2019-03-11,MSK,android,78.0,3659.61,46.918077
253,real,2019-03-11,MSK,ios,9.0,284.88,31.653333
254,real,2019-03-11,SPB,android,27.0,2314.06,85.705926


In [ ]:
# записываем результат

out.to_csv('../working/out.csv', index=False)